In [1]:
import cv2
import mediapipe as mp
import argparse
from pythonosc import udp_client

import numpy as np

cap = cv2.VideoCapture(0)
mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils


In [2]:
def normalizing_coordinates(coordinates):
    temp1, temp2 = coordinates[0,0], coordinates[0,1]
    for i in range(coordinates[0].size):
        coordinates[i,0] -= temp1
        coordinates[i,1] -= temp2
    
    coordinates = coordinates.flatten()
    max_abs = max(abs(coordinates))
    coordinates = coordinates/max_abs

    return coordinates

In [3]:
START_SOUND = True
# start the osc
# argparse helps writing user-friendly commandline interfaces
parser = argparse.ArgumentParser()
# OSC server ip: '127.0.0.1'
parser.add_argument("--ip", default='127.0.0.1', help="The ip of the OSC server")
# OSC server port (check on SuperCollider) 57120
parser.add_argument("--port", type=int, default=57120, help="The port the OSC server is listening on")

# Parse the arguments
args, unknown = parser.parse_known_args()

# Start the UDP Client
client = udp_client.SimpleUDPClient(args.ip, args.port)

In [4]:
while True:
    success, image = cap.read()
    image = cv2.flip(image, 1)
    imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(imageRGB)
    center_x = []
    center_y = []
    array_indx = np.zeros((21,2))
    # checking whether a hand is detected
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks: # working with each hand
            for id, lm in enumerate(handLms.landmark):
                h, w, c = image.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                # fill the index array to normalize and feed into the classifier network
                array_indx[id,0], array_indx[id,1] = cx, cy
                # helps to extract the center coordinates
                if(id == 0 or id == 5 or id == 17):
                    center_x.append(cx)
                    center_y.append(cy)
                
            center_palm_x = int(sum(center_x)/len(center_x))
            center_palm_y = int(sum(center_y)/len(center_y))
            cv2.circle(image, (center_palm_x, center_palm_y), 10, (255, 0 ,255), cv2.FILLED)

            mpDraw.draw_landmarks(image, handLms, mpHands.HAND_CONNECTIONS)

            normalized_coordinates = normalizing_coordinates(array_indx)
            

        
        if START_SOUND:
            client.send_message("freq", center_palm_x)
	    
            client.send_message("amp", center_palm_y)
            
    cv2.imshow("Output", image)
    cv2.waitKey(1)
    
    keypress = cv2.waitKey(1) & 0xFF		
    if keypress == ord("q"):
		    break

# free up memory
cap.release()
cv2.destroyAllWindows()